In [20]:

import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cvxpy as cp
import yfinance as yf
from datetime import datetime
from sklearn.covariance import LedoitWolf


In [ ]:
TICKERS = [
    # Large-cap tech & communication
    "AAPL","MSFT","GOOGL","AMZN","TSLA","META","NVDA","NFLX","ADBE","CRM",

    # Semiconductors
    "INTC","AMD","QCOM","AVGO","TXN","MU","AMAT","LRCX","NXPI","KLAC",

    # Banks & financials
    "JPM","BAC","WFC","C","GS","MS","V","MA","PYPL","AXP",

    # Healthcare & pharma
    "PFE","JNJ","MRK","UNH","ABBV","LLY","BMY","GILD","AMGN","REGN",

    # Energy
    "XOM","CVX","BP","SHEL","TTE","COP","SLB","HAL","MPC","PSX",

    # Consumer staples & discretionary
    "PG","KO","PEP","WMT","COST","HD","MCD","NKE","TGT","SBUX",

    # Industrials
    "BA","CAT","DE","HON","GE","UPS","MMM","LMT","RTX","FDX",

    # Utilities
    "NEE","DUK","SO","AEP","EXC","SRE","WEC","PEG","ED","XEL",

    # Telecom
    "VZ","T","TMUS","CMCSA","CHTR","DIS","FOX","FOXA","ROKU",

    # More large-caps for diversity
    "ORCL","IBM","SAP","CSCO","MMM","BKNG","SPGI","BLK","MO","PM",
    "ADSK","WDAY"
]